## Async client examples

More async client examples [here](https://apache-ignite-binary-protocol-client.readthedocs.io/en/latest/async_examples.html)

In [52]:
import asyncio
import pandas as pd

from pyignite import GenericObjectMeta, AioClient
from pyignite.datatypes import String, IntObject, DoubleObject
from pyignite.datatypes.prop_codes import PROP_NAME, PROP_SQL_SCHEMA, PROP_QUERY_ENTITIES

In [53]:
CACHE_CONFIG = {
    PROP_NAME: 'sql_public_student',
    PROP_SQL_SCHEMA: 'public',
    PROP_QUERY_ENTITIES: [
        {
            'table_name': 'student',
            'key_field_name': 'sid',
            'key_type_name': 'java.lang.Long',
            'field_name_aliases': [],
            'query_fields': [
                {
                    'name': 'sid',
                    'type_name': 'java.lang.Long',
                    'is_key_field': True,
                    'is_notnull_constraint_field': True,
                },
                {
                    'name': 'name',
                    'type_name': 'java.lang.String',
                },
                {
                    'name': 'login',
                    'type_name': 'java.lang.String',
                },
                {
                    'name': 'age',
                    'type_name': 'java.lang.Integer',
                },
                {
                    'name': 'gpa',
                    'type_name': 'java.math.Double',
                },
            ],
            'query_indexes': [],
            'value_type_name': 'sql_public_student_type',
            'value_field_name': None,
        },
    ],
}

In [54]:
class Student(
    metaclass=GenericObjectMeta,
    type_name='sql_public_student_type',
    schema={
        'name': String,
        'login': String,
        'age': IntObject,
        'gpa': DoubleObject,
    }
):
    pass

In [55]:
students = [
    {'name': 'John Doe', 'login': 'jdoe', 'age': 18, 'gpa': 2.5},
    {'name': 'Jane Doe', 'login': 'janed', 'age': 25, 'gpa': 3.5},
    {'name': 'Eva Lu Ator', 'login': 'evaluator', 'age': 45, 'gpa': 4.9}
]

In [56]:
client = AioClient(partition_aware=True)
loop = asyncio.get_event_loop()

### Cache operations example

In [57]:
async def cache_api():
    async with client.connect([('127.0.0.1', 10800 + i) for i in range(3)]):
        cache = await client.get_or_create_cache(CACHE_CONFIG)
        await cache.put_all({i: Student(**st) for i, st in enumerate(students)})


        async with client.sql('select * from student', include_field_names=True) as cursor:
            row_names = await cursor.__anext__()
            rows = [row async for row in cursor]
            return pd.DataFrame(rows, columns=row_names)


# We must use task because jupyter uses tornado and event loop is in use.
await loop.create_task(cache_api())

,SID,NAME,LOGIN,AGE,GPA
0,0,John Doe,jdoe,18,2.5
1,1,Jane Doe,janed,25,3.5
2,2,Eva Lu Ator,evaluator,45,4.9


### Scan query example

In [58]:
async def scan_query():
    async with client.connect([('127.0.0.1', 10800 + i) for i in range(3)]):
        cache = await client.get_or_create_cache(CACHE_CONFIG)
        await cache.put_all({i: Student(**st) for i, st in enumerate(students)})

        
        row_names = ['name', 'login', 'age', 'gpa', 'version']
        rows = []
        
        async with cache.scan() as cursor:
            async for k,v in cursor:
                row = [k]
                row += [getattr(v, name, None) for name in row_names]
                rows.append(row)

        return pd.DataFrame(rows, columns=['sid'] + row_names)


# We must use task because jupyter uses tornado and event loop is in use.
await loop.create_task(get_students())

,SID,NAME,LOGIN,AGE,GPA
0,0,John Doe,jdoe,18,2.5
1,1,Jane Doe,janed,25,3.5
2,2,Eva Lu Ator,evaluator,45,4.9


### Sql query example

In [60]:
async def sql_query():
    async with client.connect([('127.0.0.1', 10800 + i) for i in range(3)]):
        async with client.sql('select * from student', include_field_names=True) as cursor:
            row_names = await cursor.__anext__()
            rows = [row async for row in cursor]
            return pd.DataFrame(rows, columns=row_names)


# We must use task because jupyter uses tornado and event loop is in use.
await loop.create_task(sql_query())

,SID,NAME,LOGIN,AGE,GPA
0,0,John Doe,jdoe,18,2.5
1,1,Jane Doe,janed,25,3.5
2,2,Eva Lu Ator,evaluator,45,4.9
